# Equivariance Tutorial

In [ ]:
import torch
from torch.nn.functional import affine_grid, grid_sample, pad, relu
from torchvision.io import read_image
import lietorch
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
extent = (-1, 1, -1, 1)

In [ ]:
shoe = read_image("images/shoe.png")[None, ...] / 255.
dress = read_image("images/dress.png")[None, ...] / 255.

In [ ]:
def transform(image, x, A, padding=True):
    """Apply action (x, A) to image with circular padding."""
    B, C, H, W = image.shape
    x = torch.tensor([-1., 1.]) * x
    affine_matrix = torch.hstack((torch.linalg.inv(A.T), x[None, ...].T / 3))
    if padding:
        image = pad(image, (W, W, H, H), mode="circular")
        grid = affine_grid(affine_matrix[None, ...], (B, C, 3 * H, 3 * W), align_corners=False)
        return grid_sample(image, grid, align_corners=False)[..., H:2*H, W:2*W]
    else:
        grid = affine_grid(affine_matrix[None, ...], (B, C, H, W), align_corners=False)
        return grid_sample(image, grid, align_corners=False)

## Theory

### Lie Groups

> **Definition (Lie Group)** $G$ is a _Lie group_ if it is 
> 1. a _smooth manifold_ - so smooth and looks locally like $\mathbb{R}^n$ - and
> 2. a _group_ - we have a smooth, well-behaved product $\cdot: G \times G \to G$.

The most important Lie groups (imo) encode continuous symmetries on other spaces, with the group product simply given by composition.

> **Example (Translation Group)**
> The $n$-dimensional _translation group_ $\mathbb{R}^n$ acts on Euclidean space $\mathbb{R}^n$ by translation, namely
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}, $$
> and has group product
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}. $$
> Consequently, $\mathbb{R}^n$ also acts on the functions on Euclidean space by
> $$ (\mathbf{x}, f) \mapsto (\mathbf{y} \mapsto f(\mathbf{y} - \mathbf{x})). $$
Of course, this is an incredibly boring example. 
Slightly less trivial is the following:
> **Example (Special Orthogonal Group)**
> The _special orthogonal group_ $\operatorname{SO}(n)$ acts on Euclidean space $\mathbb{R}^n$ by rotation, namely
> $$ (R, \mathbf{y}) \mapsto R\mathbf{y}, $$
> and has group product
> $$ (R, S) \mapsto RS, $$
> Consequently, $\operatorname{SO}(n)$ also acts on the functions on Euclidean space by
> $$ (R, f) \mapsto (\mathbf{x} \mapsto f(R^{-1} \mathbf{x})). $$
Here, we represent the elements of $\operatorname{SO}(n)$ as $n \times n$ orthogonal matrices with determinant $1$. 
For example, in two dimensions the counter-clockwise rotation by angle $\theta$ is given by
$$ R = \begin{pmatrix} \cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \end{pmatrix}. $$

In this example the group and the space that is acted on can no longer be identified.

We get our favourite group by combining the two previous ones:
> **Example (Special Euclidean Group)**
> The _special Euclidean group_ $\operatorname{SE}(n)$ acts on Euclidean space $\mathbb{R}^n$ by roto-translation, namely
> $$ ((\mathbf{x}, R), \mathbf{y}) \mapsto \mathbf{x} + R\mathbf{y}, $$
> and has group product
> $$ ((\mathbf{x}, R), (\mathbf{y}, S)) \mapsto (\mathbf{x} + R\mathbf{y}, RS). $$
> Consequently, $\operatorname{SE}(n)$ also acts on the functions on Euclidean space by
> $$ (R, f) \mapsto (\mathbf{y} \mapsto f(R^{-1}(\mathbf{y} - \mathbf{x}))). $$

> _Remark_ The actions on functions are examples of so-called _group representations_, a term often encountered in the equivariance literature. For the sake of simplicity here we simply refer to them as actions.

Finally, we have the most simple group action: doing nothing:
> **Definition (Trivial Action)** Let $G$ be a Lie group and $X$ a set. Then we $G$ acts _trivially_ on $X$ if $g x = x$ for all $g \in G$, $x \in X$.

Many problems have inherent symmetries. For example, if we want to classify the object in an image, rotating the object shouldn't change the classification: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_title("Shoe")
ax[1].set_title("Still a shoe")

x = torch.tensor([0., 0.])
R = torch.tensor([[1., 0.], [0., 1.]])
ax[0].imshow(transform(shoe, x, R).squeeze(), extent=extent)

# Your roto-translation here ⬇️
x = torch.tensor([0., 0.])
theta = torch.tensor([0.])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])
ax[1].imshow(transform(shoe, x, R).squeeze(), extent=extent);

Lie groups give us a mathematically formal way of thinking about these symmetries. In particular, we can now define _equivariance_, which is the formal way of describing that an operator preserves a symmetry.

### Equivariance

> **Definition (Equivariance)** Let $G$ be a Lie group acting on $U$ and $V$. 
> $\Phi: U \to V$ is called _equivariant_ if it commutes with the group actions, i.e.
> $$ \Phi \circ g = g \circ \Phi, $$
> for all $g \in G$.

If the action on $V$ is trivial, then we say $\Phi$ is _invariant_. 

Let's work this out for our classification problem.
- We have a classifier $\Phi: \mathbb{L}_2(\mathbb{R}^2) \to \{1, \ldots, c\}$, where $c$ is the number of classes.
- We can see images as functions on $\mathbb{R}^2$, on which the Lie group $\operatorname{SE}(2)$ acts by roto-translation: $(\mathbf{x}, R) f(\vec{y}) = f(R^{-1} (\mathbf{y} - \mathbf{x}))$.
- $\operatorname{SE}(2)$ acts trivially on the range $\{1, \ldots, c\}$, so we have $(\mathbf{x}, R) k = k$ for all $k \in \{1, \ldots, c\}$.

Then the classifier is invariant if $\Phi(f) \circ g = g \circ \Phi(f) = \Phi(f)$ for all $g \in \operatorname{SE}(2)$ and images $f: \mathbb{R}^2 \to \mathbb{R}$.

In this problem invariance is clearly a desirable property. But how would we go about constructing an invariant classifier? 

We could train a normal convolutional neural network, and hope that it learns to be invariant. This is highly unlikely, unless we perform _data augmentation_ - - and even then there are no guarantees.

Alternatively, we could construct a neural network architecture that is inherently invariant. For this, we can make use of the following result:
> **Lemma (Composition of Equivariant Maps)** Let $G$ be a Lie group acting on $U$, $V$, and $W$. Suppose $\Phi: U \to V$ and $\Psi: V \to W$ are equivariant. Then, their composition $\Psi \circ \Phi: U \to W$ is also equivariant. 

_proof_: Simply note that $\Psi \circ \Phi \circ g = \Psi \circ g \circ \Phi = g \circ \Psi \circ \Phi$.

Hence, we can make an equivariant neural network architecture by composing equivariant layers. A typical layer in a neural network consists of the composition of something linear (e.g. matrix multiplication, convolution, linear combinations) with something nonlinear (e.g. activation function, normalisation). Common nonlinearities such as the ReLU activation function and batch normalisation act point-wise; it is not hard to see that these are equivariant operations.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("ReLUed Shoe")
ax[1].set_title("Still a ReLUed shoe")
ax[0].imshow(relu(shoe - 0.5).squeeze())
x = torch.tensor([0., 0.])
theta = torch.tensor([-0.862364]) # Random number
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])
ax[1].imshow(relu(transform(shoe, x, R, padding=False) - 0.5).squeeze());

### Lifting

## Application